# Chapter 08 시계열 특징의 생성 및 선택

## 8.4 생성된 특징들 중 일부를 선택하는 방법

tsfresh를 통해 FRESH 알고리즘 구현

In [4]:
!pip install tsfresh

In [9]:
# 로봇 실행 실패 관련 시계열 데이터 다운로드
import tsfresh
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, \
                                                        load_robot_execution_failures

download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()

In [17]:
import pandas as pd
import numpy as np

### 자동으로 특징 추출
계산 자원을 고려하지 않은 채 포괄정으로 특징을 생성하기 때문에 문제가 있음.<br/>
실제로는 규모가 작은 데이터셋일지라도 적당한 크기로 축소하지 않은 채 무턱대고 배포해서는 안됨

In [20]:
from tsfresh import extract_features

extracted_features = extract_features(timeseries, column_id = "id", column_sort = "time")

Feature Extraction: 100%|██████████| 528/528 [00:27<00:00, 19.48it/s]


tsfresh는 원하는 특징을 지정할 수 있지만, 여기서는 모든 특징 포함<br/>

In [22]:
extracted_features.columns

Index(['F_x__variance_larger_than_standard_deviation',
       'F_x__has_duplicate_max', 'F_x__has_duplicate_min',
       'F_x__has_duplicate', 'F_x__sum_values', 'F_x__abs_energy',
       'F_x__mean_abs_change', 'F_x__mean_change',
       'F_x__mean_second_derivative_central', 'F_x__median',
       ...
       'T_z__permutation_entropy__dimension_6__tau_1',
       'T_z__permutation_entropy__dimension_7__tau_1',
       'T_z__query_similarity_count__query_None__threshold_0.0',
       'T_z__matrix_profile__feature_"min"__threshold_0.98',
       'T_z__matrix_profile__feature_"max"__threshold_0.98',
       'T_z__matrix_profile__feature_"mean"__threshold_0.98',
       'T_z__matrix_profile__feature_"median"__threshold_0.98',
       'T_z__matrix_profile__feature_"25"__threshold_0.98',
       'T_z__matrix_profile__feature_"75"__threshold_0.98',
       'T_z__mean_n_absolute_max__number_of_maxima_7'],
      dtype='object', length=4734)

### RFE는 특징 선택과 중요성의 순위를 매기는 방법 모두에 사용가능
이를 시험해 보기위해 FRESH 알고리즘으로 고른 특징 목록 중 10개,<br/>
FRESH 알고리즘이 기각한 특징 목록 중 10개를 무작위로 선택하여 결합

In [28]:
# FRESH 알고리즘이 고른 특징을 알아낼줄 몰라서 일단
# 임의의 index를 가지고 실행함
import random

x_idx = random.sample(range(len(extracted_features)), 20)
mixed_X = extracted_features.iloc[:, x_idx].values

20개의 특징에 대해 RFE를 수행하면, RFE에 사용되는 모델에 대한 중요도 순위를 반환

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

svc = SVC(kernel="linear", C=1)
rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
rfe.fit(mixed_X, y)
rfe.ranking_